In [18]:
# Set and plot the data
import matplotlib.pyplot as plt

data = open("rectangle.txt", "r")
X = []
Y = []
labels = []
colors = ["blue", "red"]
shapes = ["o","^"]

for line in data:
    tmp = line.split()
    X.append(float(tmp[0]))
    Y.append(float(tmp[1]))
    labels.append(int(tmp[2]))

data.close()

"""
CREATE LINE EQUATION FROM TWO POINTS
INPUT - x1,y1,x2,y2
OUTPUT - m,n (such represent y=mx+n)
"""
def lineEquation(x1,y1,x2,y2):
    if np.abs(x1-x2) < 0.0001:
        m = None
        n = x1
    else:
        m = (y1-y2)/(x1-x2)
        n = y1 - (m*x1)
    return m,n

# Methods to plot the ponits on graph
"""
plot the points on graph (without any line)
"""
def plotSet(X,Y,labels):
    for p in range(len(X)):
        index = max(0,labels[p])
        color = colors[index]
        shape = shapes[index]
        xp = X[p]
        yp = Y[p]
        plt.scatter(xp,yp,c = color,marker = shape)

    plt.grid()
    plt.show()
"""
plot the points on graph (without one line)
"""
def plotSetWithLine(X,Y,labels,m,n):
    for p in range(len(X)):
        index = max(0,labels[p])
        color = colors[index]
        shape = shapes[index]
        xp = X[p]
        yp = Y[p]
        plt.scatter(xp,yp,c = color,marker = shape, linewidths=2)
    plt.grid()
    if m is not None:
        x = np.linspace(-4,4,100)
        y = m*x+n
        plt.plot(x, y, '-g')
    else:
        plt.plot([n,n], [0, 4], color='black', linestyle='-', linewidth=1)
    plt.xlim([0, 4])
    plt.ylim([0, 4])
    plt.show()
    
"""
plot the points on graph (without k lines)
"""   
def plotExample(rules, setPoints):
    dataSet = list(zip(*setPoints))
    
    for p in range(len(dataSet[0])):
        index = max(0,dataSet[2][p])
        color = colors[index]
        shape = shapes[index]
        xp = dataSet[0][p]
        yp = dataSet[1][p]
        plt.scatter(xp,yp,c = color,marker = shape, linewidths=2)
        
    plt.grid()
    
    for x1,y1,x2,y2,alpha,flag in rules:
        m,n = lineEquation(x1,y1,x2,y2)
        if m is not None:
            x = np.linspace(-4,4,100)
            y = m*x+n
            plt.plot(x, y, '-g', linestyle='-', linewidth = 1)
        else:
            plt.plot([n,n], [0, 4], color='black', linestyle='-', linewidth=1)
            
    plt.xlim([0, 4])
    plt.ylim([0, 4])
    plt.show()    

In [19]:
#Set pre-process data
import random as rnd

"""
SPLIT DATA 50-50, RETURN S(=TRAIN) AND T(=TEST)
"""
def splitData5050(X,Y,labels):
    S = [] 
    T = []
    for i in range(len(X)):
        if (rnd.getrandbits(1) == 1 and len(S)<(len(X)/2)) or len(T) >=(len(X)/2) :
            S.append((X[i],Y[i],labels[i]))
        else:
            T.append((X[i],Y[i],labels[i]))
        
            
    return S,T

In [20]:
import numpy as np

"""
Check the sign by k rules
"""
def getSign (k_rules, k, xi,yi ):
    sigma = 0
    for i in range(k):
        m,n = lineEquation(k_rules[i][0],k_rules[i][1],k_rules[i][2],k_rules[i][3])
        val = predictHi(xi,yi,m,n)
        
        # opposite sign 
        if k_rules[i][5] == "neg":
            val = val * (-1)
            
        sigma = sigma + k_rules[i][4] * val
    
    if(sigma > 0):
        return 1
    else:
        return -1
    
"""
Get the predict value of (xi,yi) given the m,n of rule
"""
def predictHi(xi,yi,m,n):
    if m is not None:
        if ((m*xi)+n)-yi <= 0:
            return 1 # above the line
        else:
            return -1; # under the line
    else:
        if xi > n:
            return 1
        else:
            return -1
    
"""
Calculate the error of set of points
"""
def calError(setPoints, k_rules, k):
    sumOfErrors = 0
    for xi,yi,labeli in setPoints:
        sign = getSign(k_rules,k,xi,yi)
        if sign != labeli:
            sumOfErrors = sumOfErrors + 1
            
    return sumOfErrors / len(setPoints)        
    
#Ada-boost
def adaBoost(S):
    #Set initial weights to 1/N
    weight = [1/len(S)] * len(S)
    
    k = 8 
    #Set all rule
    H=[] 
    for h in range(len(S)):
        for i in range(h+1,len(S)): 
            x1 = S[h][0] 
            y1 = S[h][1]
            x2 = S[i][0]
            y2 = S[i][1]
            H.append((x1,y1,x2,y2))
       
    K_rules=[] # 8 rules, each rule contain (x1,y1,x2,y2,alphat,flag)
    
    for j in range(k):
        eth_pos_array = [] # hold 2775 empirical errors for each positive rule in iteration j
        eth_neg_array = [] # hold 2775 empirical errors for each negative rule in iteration j
        hj_xi = [] # 3D array - (rule, index, point)
        
        # Check for each rule the empirial error
        for x1,y1,x2,y2 in H:
            hj_pos = []
            hj_neg = []
            m,n = lineEquation(x1,y1,x2,y2)
            h_xi = 0
            eth_pos = 0
            eth_neg = 0
            if m is not None:
                for xi,yi,labeli in S:
                    if ((m*xi)+n)-yi < 0:
                        h_xi = 1 # above the line
                    else:
                        h_xi = -1; # under the line
                    
                    if h_xi != labeli:
                        eth_pos = eth_pos + weight[S.index((xi,yi,labeli))]
                    else:
                        eth_neg = eth_neg + weight[S.index((xi,yi,labeli))]
                    hj_pos.append(h_xi)
                    hj_neg.append(h_xi*-1)
            else:
                for xi,yi,labeli in S:
                    if xi > n:
                        h_xi = 1 # Right the line
                    else:
                        h_xi = -1 # left the line
                    
                    if h_xi != labeli:
                        eth_pos = eth_pos + weight[S.index((xi,yi,labeli))]
                    else:
                        eth_neg = eth_neg + weight[S.index((xi,yi,labeli))]
                    hj_pos.append(h_xi)
                    hj_neg.append(h_xi*-1)
            
            hj_xi.append([hj_pos,hj_neg])    
            eth_pos_array.append(eth_pos)
            eth_neg_array.append(eth_neg)


        # Collect the min empirical error 
        et_ht_pos = min(eth_pos_array)
        et_ht_neg = min(eth_neg_array)
        ht = 0
        flag = ""
        # Collect the index of min empirical error
        if et_ht_pos < et_ht_neg:
            flag = "pos"
            ht = eth_pos_array.index(et_ht_pos)
        else:
            flag = "neg"
            ht = eth_neg_array.index(et_ht_neg)
            
        index = 0
        if flag == "neg":
            index = 1
                   
        # Collect the alpha
        alphat = 0
        if flag == "neg" : 
            alphat = 0.5 * np.log((1-et_ht_neg)/(et_ht_neg))
        else:
            alphat = 0.5 * np.log((1-et_ht_pos)/(et_ht_pos))

        K_rules.append([H[ht][0],H[ht][1],H[ht][2],H[ht][3], alphat,flag])
        
        # Update the weights
        l = np.array(hj_xi)
        for i in range(len(weight)):
            index = 0
            if flag == "neg":
                index = 1  
                
            ht_xi = hj_xi[ht][index][i]
            weight[i] = weight[i] * np.exp(-alphat*ht_xi*S[i][2])
        
        # Normalize the weights
        sum_weights = 0
        for i in weight:
            sum_weights = sum_weights + i

        for i in range(len(weight)):
            weight[i] = weight[i]/sum_weights
        
    return K_rules       

In [ ]:
"""
Check the error on set points by k rules
"""
def check_error(set_points, k_rules):
    errors = []
    for i in range(len(k_rules)):
        count = 0
        for p in set_points:
            pred = getSign(k_rules , i , p[0] , p[1])               
            if pred != p[2]:
                count = count + 1
        errors.append(count/len(set_points))
    return errors

emp_error = [0]*8
true_error = [0]*8

# Run Adaboost # times
times = 100
res_example = []
res_S = []
res_T = []
for i in range(times):
    # Split the data
    S,T = splitData5050(X,Y,labels)
    # Run adaboost on train set
    res = adaBoost(S)

    # Check the empirical error on train
    emp_err_i = check_error(S, res)
    # Check the true error on test
    true_err_i = check_error(T, res)
    
    # Add values to the main arrays
    for j in range(8):
        emp_error[j] = emp_error[j] + emp_err_i[j]
        true_error[j] = true_error[j] + true_err_i[j]

# Extract the mean of # times
avg_empirical_error = [element/times for element in emp_error]         
avg_true_error = [element/times for element in true_error] 


print("AVG EMPIRICAL ERROR\n", avg_empirical_error)
print("AVG TRUE ERROR\n", avg_true_error)

In [ ]:
# Plot the graph that represent the difference between avg train error and avg test error

plt.plot(avg_empirical_error, label="mean train error")
plt.plot(avg_true_error, label="mean true error")
plt.xlabel("numbers of rules")
plt.ylabel("errors in %")
plt.legend()
plt.show()